The ultimate goal is to perform holoVQE. \
It is divided into two parts: first part include functions to calculate expectation value of H; the second part include functions for the optimization\
More specifically, part 1 includes: functions to construct MPS from the unitary matrices, functions to construct H as MPO and functions to calculate the expectation value of H.\
Part 1:

In [167]:
def Uni_to_ten(uni_matrix, site_num, d_bond, d_phys_index, L):
    '''
    construct the matrix corresponding to physics leg index in local Hilbert space for translational invariant 1d chain
    
    Parameters
    ----------
    uni_matrix: numpy.ndarray
        the unitary matrix already constructed
    site_num: int
        the position of the site
    d_bond: int
        bond dimension
    d_phys_index: int
        the index in the physical dimension
    L: int
        the total number of sites
    '''
    if site_num == 0:
        res_matrix = uni_matrix[0:1, d_phys_index * d_bond: (d_phys_index+1)*d_bond]
    elif site_num == L-1:
        res_matrix = uni_matrix[0:d_bond, d_phys_index * d_bond: d_phys_index * d_bond + 1]
    else:
        res_matrix = uni_matrix[0:d_bond, d_phys_index * d_bond: (d_phys_index + 1) * d_bond]
    return res_matrix

In [169]:
import tenpy.linalg.np_conserved as npc
def Site_Ten(uni_matrix, site_num, d_bond, d_phys, L):
    '''
    construct the 3-leg tensor for one site of the translational invariant 1d chain
    
    Parameters
    ----------
    d_phys: dimension of local Hilbert space
    
    returns the npc.Array type, which contains the tensor and its leg labels
    '''
    res_tensor = npc.Array.from_ndarray_trivial([Uni_to_ten(uni_matrix, site_num, d_bond, i, L) \
                                                 for i in range(0,d_phys)],labels=['p'+str(site_num),'vL','vR'])
    return res_tensor

In [170]:
from tenpy.networks.mps import MPS
from tenpy.networks.site import BosonSite
from tenpy.models.lattice import Chain
import tenpy.linalg.np_conserved as npc

def MPS_state(UM_list, d_phys, d_bond, L):
    '''
    construct the Matrix Product state for the whole chain (1d translational invariant bosonic chain) (can be adjusted)
    
    Parameters
    ----------
    UM_list: list of numpy.ndarray matrices
    
    returns the MPS type in TenPy
    '''
    site = BosonSite()
    lat = Chain(L, site, bc_MPS = 'finite')
    state = Site_Ten(UM_list[0], 0, d_phys, d_bond, L)
    for i in range(1, L):
        state1 = Site_Ten(UM_list[i], i, d_phys, d_bond, L)
        state = npc.tensordot(state, state1,('vR','vL'))
    psi = MPS.from_full(lat.mps_sites(), state)
    # other methods to consider, MPS.from_Bflat, MPS
    # problem with MPS.from_Bflat now
    return psi

In [171]:
from tenpy.networks.mpo import MPO
# construct Hamiltonian as a MPO
# H_mpo = MPO(sites, Ws)
# Ws: lists of Array: matrices of MPO

In [172]:
from tenpy.networks.mpo import MPO, MPOEnvironment
def MPO_H_contract(H_mpo, psi, L):
    '''
    calculate the expectation value of H by doing the contraction over the full MPS

    Parameters
    ----------
    H_mpo: MPO type
    psi: MPS type
    L: integer
        the total number of sites
    '''
    env = MPOEnvironment(psi, H_mpo, psi)
    E = env.full_contraction(L-1)
    return E

tests:

In [178]:
import tenpy
import numpy as np
import tenpy.linalg.np_conserved as npc
from scipy.stats import unitary_group

# for purpose of test, generate arbitrary unitary matrices
x1 = unitary_group.rvs(4)
x2 = unitary_group.rvs(4)
# test Uni_to_ten
x1_1 = Uni_to_ten(x,0,2,1,2)

# test Site_ten
x1_t = Site_Ten(x1,0,2,2,2)
x2_t = Site_Ten(x2,1,2,2,2)
x1_a = npc.Array.to_ndarray(x1_t)
x2_a = npc.Array.to_ndarray(x2_t)

print(x1)
print('******')
print(x2)
print('******')
print(x1_1)
print('******')
print(x1_t.rank)
print(x1_a)
print('******')
print(x2_a)

<class 'numpy.ndarray'>
[[ 0.38543826+0.00287978j -0.28343129+0.76121007j  0.07968152-0.06270453j
   0.39198005+0.16651012j]
 [ 0.36545898+0.64769324j  0.11825688-0.06416217j  0.38425936-0.17738365j
  -0.04925721-0.49727782j]
 [-0.11996199-0.23038543j -0.53150838+0.08400989j -0.23434959+0.14047791j
  -0.03786807-0.75291772j]
 [ 0.40664976-0.25598807j  0.03906638-0.1761802j   0.36212729+0.77769456j
   0.02426973-0.00253162j]]
******
[[-0.16659523+0.07528551j -0.34309477+0.06784042j -0.82169032-0.24328563j
   0.23861959-0.2301295j ]
 [-0.30650742+0.13058848j  0.79270562-0.26116851j -0.35873116-0.06218596j
  -0.17812002+0.1677104j ]
 [ 0.75897492-0.52251143j  0.16512909-0.12697881j -0.31156014-0.0270706j
  -0.08674267+0.04712551j]
 [-0.02611358-0.07638673j  0.13038168-0.34742171j  0.1686338 -0.08271686j
  -0.0815699 -0.9021357j ]]
******
[[ 0.02632271-0.18775696j -0.24329483+0.5502272j ]]
******
3
[[[ 0.38543826+0.00287978j -0.28343129+0.76121007j]]

 [[ 0.07968152-0.06270453j  0.39198005

In [175]:
# test for MPS.from_full
from tenpy.networks.site import BosonSite
from tenpy.models.lattice import Chain
import tenpy.linalg.np_conserved as npc
site = BosonSite()
L = 2
lat = Chain(L, site, bc_MPS = 'finite')
state = npc.tensordot(x_t, x_t1, ('vR','vL'))
psi = MPS.from_full(lat.mps_sites(), state)

print(site)
print(lat)
print(type(state))
print(type(psi))
print(psi)

BosonSite(1, 'N', 0.000000)
<class 'tenpy.linalg.np_conserved.Array'>
<class 'tenpy.networks.mps.MPS'>
MPS, L=2, bc='finite'.
chi: [2]
sites: BosonSite(1, 'N', 0.000000) BosonSite(1, 'N', 0.000000)
forms: (1.0, 0.0) (0.0, 1.0)


In [176]:
# test MPS_state function
print(MPS_state([x,x11], 2, 2, 2))

MPS, L=2, bc='finite'.
chi: [2]
sites: BosonSite(1, 'N', 0.000000) BosonSite(1, 'N', 0.000000)
forms: (1.0, 0.0) (0.0, 1.0)
